### Number Plate Recognizer

In [ ]:
! sudo apt install tesseract-ocr
! pip install pytesseract  --quiet
! pip install streamlit    --quiet
! pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok
! pip install opencv-python
! wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,816 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
%%writefile app.py
import streamlit as st
import pytesseract
import numpy as np
import cv2
from PIL import Image
st.title("Number Plate Recognizer")
img = st.sidebar.file_uploader("Choose an image")
if img is not None :
  img_read = Image.open(img)
  st.image(img)
  if st.button('PREDICT') :
    classifier = cv2.CascadeClassifier('/content/haarcascade_russian_plate_number.xml')
    img = np.array(img_read)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    carplate_x = img_rgb
    carplate_rects = classifier.detectMultiScale(carplate_x,scaleFactor=1.1, minNeighbors=7)
    for x,y,w,h in carplate_rects: 
      cv2.rectangle(carplate_x, (x,y), (x+w,y+h), (255,0,0), 5)
    st.image(carplate_x)
    carplate_rects = classifier.detectMultiScale(img_rgb,scaleFactor=1.6, minNeighbors=7)
    for x,y,w,h in carplate_rects: 
      carplate_img = img_rgb[y+15:y+h-10 ,x+15:x+w-20]
    carplate_img_gray = cv2.cvtColor(carplate_img, cv2.COLOR_RGB2GRAY)
    carplate_img_gray_blur = cv2.medianBlur(carplate_img_gray,3)
    op = pytesseract.image_to_string(carplate_img_gray_blur)
    a = []
    text = " "
    for i in op :
      if ( i  in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890') :
        a.append(i)
    listToStr = ' '.join(map(str, a))

    st.text('The predicted number plate - ')

    st.write(listToStr)

Writing app.py


In [ ]:
! nohup streamlit run app.py &
url = ngrok.connect(port = '8501')
print(url)

nohup: appending output to 'nohup.out'
http://10e0c8c82636.ngrok.io
